# 3D Mnist - machine learning project

Author: Mateusz Sabuk

## Imports

In [1]:
import h5py
import pickle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

2024-06-03 23:52:22.502419: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-03 23:52:22.621908: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-03 23:52:23.078345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 23:52:24.559825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Functions

- create_model(variables) -> model
- train_model(model, variables) -> training_history
- plot_model(model, training_history, model_name): saves plots at ./output/
- save_data_to_file(data, data_name="data"): saves data to file at ./output/

In [10]:
def create_model(variables):
    model = Sequential()

    model.add(Input(shape=(16, 16, 16, 1)))

    # Create convolutional layers
    for filter_num, kernel_size, pool_size in zip(variables["filters"], variables["kernel_sizes"], variables["pool_sizes"]):
        model.add(Conv3D(filters=filter_num, kernel_size=kernel_size, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=pool_size))

    # Flatten the output from the convolutional layers
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(units=variables["dense"], activation='relu'))
    model.add(Dropout(variables["dropout"]))  # Adding dropout for regularization

    # Output layer
    model.add(Dense(units=10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=variables["learning_rate"]), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def train_model(model, variables):
    return model.fit(X_train, y_train, epochs=variables["epochs"], batch_size=variables["batch_size"], validation_data=(X_test, y_test))

def plot_model(model, training_history, model_name):
    # Plot training & validation accuracy values
    plt.figure(figsize=(8, 6))
    plt.plot(training_history.history['accuracy'])
    plt.plot(training_history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'])
    plt.savefig(f'output/{model_name}_accuracy.png', bbox_inches='tight')

    # Plot training & validation loss values
    plt.figure(figsize=(8, 6))
    plt.plot(training_history.history['loss'])
    plt.plot(training_history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'])
    plt.savefig(f'output/{model_name}_accuracy.png', bbox_inches='tight')

    # Predict the classes on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = y_pred.argmax(axis=1)
    # Compute the confusion matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    # Display the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
    disp.plot(cmap=plt.cm.Reds)
    plt.savefig(f'output/{model_name}_cm.png', bbox_inches='tight')

def save_data_to_file(data, data_name="data"):
    with open(f'output/{data_name}.pkl', 'wb') as f:
        pickle.dump(data, f)

### Unused ones

I wanted to save the models with checkpoints but it was not really the part of the project

In [3]:
# def show_available_models():
#     matching_filenames = []
#     for filename in os.listdir('models/'):
#         if filename.endswith(f".h5"):
#             matching_filenames.append(filename)

#     for filename in matching_filenames:
#         # Access the metadata
#         with h5py.File(filename, 'r') as f:
#             if 'metadata' in f:
#                 metadata = f['metadata'].attrs
#                 variables = metadata.get('variables', '')
#                 print(f'File: {filename}')
#                 print(f'Used variables: {variables}')


# def get_model_from_file(model_name):
#     return load_model(f"models/{model_name}.h5")


# def save_model_to_file(model_name, variables):
#     # Load the best model
#     model = load_model(f'models/{model_name}.keras')

#     # Save the model again with additional metadata
#     model_save_path = f'models/{model_name}.h5'
#     model.save(model_save_path)

#     # Open the saved model file to add metadata
#     with h5py.File(model_save_path, 'a') as f:
#         # Create a group for metadata
#         metadata_group = f.create_group('metadata')
#         # Add descriptions or any other metadata
#         metadata_group.attrs['variables'] = variables

## Get data from h5 file

In [4]:
with h5py.File("data/full_dataset_vectors.h5", "r") as hf:
    X_train_vec = hf["X_train"][:]
    y_train = hf["y_train"][:]
    X_test_vec = hf["X_test"][:]
    y_test = hf["y_test"][:]

X_train_vec.shape, y_train.shape, X_test_vec.shape, y_test.shape

((10000, 4096), (10000,), (2000, 4096), (2000,))

### Reshape from 4096 vectors to 16x16x16 with 1 channel

In [5]:
X_train = X_train_vec.reshape(-1, 16, 16, 16, 1)
X_test = X_test_vec.reshape(-1, 16, 16, 16, 1)

X_train.shape, X_test.shape

((10000, 16, 16, 16, 1), (2000, 16, 16, 16, 1))

## Model variables

variables example:
```python3
model_variables = {
    # Convolutional layers
    "filters": [32, 64, 128],
    "kernel_sizes": [2, 2, 2],
    "pool_sizes": [2, 2, 2],
    # Rest of the model
    "dense": 512,
    "dropout": 0.3,
    "learning_rate": 0.01,
    # Model training
    "epochs": 20,
    "batch_size": 32,
}
```

In [6]:
variables_array = [{
    # Convolutional layers
    "filters": [32, 64, 128],
    "kernel_sizes": [2, 2, 2],
    "pool_sizes": [2, 2, 2],
    # Rest of the model
    "dense": 512,
    "dropout": 0.3,
    "learning_rate": 0.01,
    # Model training
    "epochs": 20,
    "batch_size": 32,
},{
    # Convolutional layers
    "filters": [128, 64, 32],
    "kernel_sizes": [2, 2, 2],
    "pool_sizes": [2, 2, 2],
    # Rest of the model
    "dense": 512,
    "dropout": 0.3,
    "learning_rate": 0.01,
    # Model training
    "epochs": 20,
    "batch_size": 32,
},{
    # Convolutional layers
    "filters": [64, 64, 64],
    "kernel_sizes": [2, 2, 2],
    "pool_sizes": [2, 2, 2],
    # Rest of the model
    "dense": 512,
    "dropout": 0.3,
    "learning_rate": 0.01,
    # Model training
    "epochs": 20,
    "batch_size": 32,
},{
    # Convolutional layers
    "filters": [32, 32, 32],
    "kernel_sizes": [2, 2, 2],
    "pool_sizes": [2, 2, 2],
    # Rest of the model
    "dense": 512,
    "dropout": 0.3,
    "learning_rate": 0.01,
    # Model training
    "epochs": 20,
    "batch_size": 32,
},
]

In [12]:
data = []
for i, variables in enumerate(variables_array):
    try:
        model = create_model(variables)
        training_history = train_model(model, variables)
        plot_model(model, training_history, str(i).zfill(2))
        data.append(tuple([variables, training_history]))
    except Exception as e:
        print(e)
        data.append(f"Model nr {str(i).zfill(2)} error: {e}")
    save_data_to_file(data)

Epoch 1/20
159/313 ━━━━━━━━━━━━━━━━━━━━ 41s 270ms/step - accuracy: 0.2015 - loss: 2.8698

KeyboardInterrupt: 